In [ ]:
import torch
import torch.nn as nn
import numpy as np
import sys
import os
import random
import matplotlib.pyplot as plt

In [ ]:
# check if GPU is available and set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


### Model + Helper Functions

In [ ]:
os.chdir("src")
print("Current working directory set to:", os.getcwd())

Current working directory set to: /projects/bcnx/sroy6/One_pixel/one-pixel-attack-pytorch/src


In [ ]:
# Please put the src directory in the path
src_path = os.path.abspath(os.path.join(os.getcwd(), 'src'))
if src_path not in sys.path:
    sys.path.append(src_path)

from utils import MIMONetDataset, ChannelScaler
from mimonet import MIMONet

stty: 'standard input': Inappropriate ioctl for device


In [ ]:
# set working directory
working_dir = "/projects/bcnx/kazumak2/MIMONet/HeatExchanger"
data_dir = os.path.join(working_dir, "data")

### Load datasets

In [ ]:
# trunk dataset
trunk_input = np.load(os.path.join(data_dir, "share/trunk.npz"))['trunk']

# min-max scaling [-1, 1]
trunk_input[:, 0] = 2 * (trunk_input[:, 0] - np.min(trunk_input[:, 0])) / (np.max(trunk_input[:, 0]) - np.min(trunk_input[:, 0])) - 1
trunk_input[:, 1] = 2 * (trunk_input[:, 1] - np.min(trunk_input[:, 1])) / (np.max(trunk_input[:, 1]) - np.min(trunk_input[:, 1])) - 1

In [ ]:
# branch input dataset
branch = np.load(os.path.join(data_dir, "branch.npz"))

branch1 = branch['branch1']
branch2 = branch['branch2']

print("Branch1 shape:", branch1.shape)
print("Branch2 shape:", branch2.shape)

# split the dataset into training and testing sets
train_size = int(0.8 * len(branch1))
test_size = len(branch1) - train_size
train_branch1, test_branch1 = branch1[:train_size], branch1[train_size:]
train_branch2, test_branch2 = branch2[:train_size], branch2[train_size:]

Branch1 shape: (1546, 2)
Branch2 shape: (1546, 100)


In [ ]:
# create a dictionary for the output channel names
# 0: turb-kinetic-energy
# 1: pressure
# 2: temperature
# 3: z-velocity
# 4: y-velocity
# 5: x-velocity
# 6: velocity-magnitude

dict_channel = {
    0: 'turb-kinetic-energy',
    1: 'pressure',
    2: 'temperature',
    3: 'z-velocity',
    4: 'y-velocity',
    5: 'x-velocity',
    6: 'velocity-magnitude'
}

# select the output channel
target_channel = [1, 3, 4, 5, 6]

# print the selected output channel names
# target_label is used to store the names of the selected output channels for further processing (e.g., plotting)
print("Selected output channels:")
target_label = []
for channel in target_channel:
    print(dict_channel[channel])
    target_label.append(dict_channel[channel])    

Selected output channels:
pressure
z-velocity
y-velocity
x-velocity
velocity-magnitude


In [ ]:
# target dataset
target = np.load(os.path.join(data_dir, "target.npy"))

# extract the output channels
target = target[:, :, target_channel ]  # select the first 7 channels
print("Target dataset shape before split:", target.shape)


# split the target dataset into training and testing sets
train_target = target[:train_size]
test_target = target[train_size:]

print("Train target shape:", train_target.shape)
print("Test target shape:", test_target.shape)

Target dataset shape before split: (1546, 3977, 5)
Train target shape: (1236, 3977, 5)
Test target shape: (310, 3977, 5)


In [ ]:
# get the mean and standard deviation of each channel
mean_branch1 = np.mean(train_branch1, axis=0)
std_branch1 = np.std(train_branch1, axis=0)

print("Mean of branch1:", mean_branch1)
print("Std of branch1:", std_branch1)

# (# train samples, 100)
mean_branch2 = np.mean(train_branch2)
std_branch2 = np.std(train_branch2)

print("Mean of branch2:", mean_branch2)
print("Std of branch2:", std_branch2)

Mean of branch1: [  4.51454429 292.42944177]
Std of branch1: [ 0.2615285  17.03323994]
Mean of branch2: 12587.66968713018
Std of branch2: 6302.709013835411


In [ ]:
# normalize the branch data using the mean and std
train_branch_1 = (train_branch1 - mean_branch1) / std_branch1
test_branch_1 = (test_branch1 - mean_branch1) / std_branch1
train_branch_2 = (train_branch2 - mean_branch2) / std_branch2
test_branch_2 = (test_branch2 - mean_branch2) / std_branch2

# print the shapes of the normalized data
print("Shape of normalized train_branch1:", train_branch_1.shape)
print("Shape of normalized test_branch1:", test_branch_1.shape)
print("Shape of normalized train_branch2:", train_branch_2.shape)
print("Shape of normalized test_branch2:", test_branch_2.shape)

Shape of normalized train_branch1: (1236, 2)
Shape of normalized test_branch1: (310, 2)
Shape of normalized train_branch2: (1236, 100)
Shape of normalized test_branch2: (310, 100)


### Scaling the target data

In [ ]:
import numpy as np
# scaling the target data
'''  
note: reverse the scaling for the target data
train_target = scaler.inverse_transform(train_target_scaled)
test_target = scaler.inverse_transform(test_target_scaled)
'''
scaler = ChannelScaler(method='minmax', feature_range=(-1, 1))
scaler.fit(train_target)
train_target_scaled = scaler.transform(train_target)
test_target_scaled = scaler.transform(test_target)

print("Shape of scaled train_target:", train_target_scaled.shape)
print("Shape of scaled test_target:", test_target_scaled.shape)

Shape of scaled train_target: (1236, 3977, 5)
Shape of scaled test_target: (310, 3977, 5)


In [ ]:
# test dataset and dataloader
test_dataset = MIMONetDataset(
    [test_branch_1, test_branch_2],  # branch_data_list
    trunk_input,                     # trunk_data
    test_target_scaled               # target_data
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=1,  # set to 1 for testing
    shuffle=False,
    num_workers=0
)

In [ ]:
# test dataset and dataloader
train_dataset = MIMONetDataset(
    [train_branch_1, train_branch_2],  # branch_data_list
    trunk_input,                     # trunk_data
    train_target_scaled               # target_data
)

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=1,  # set to 1 for testing
    shuffle=False,
    num_workers=0
)

## Model Instance & Load Parameters

In [ ]:
# Architecture parameters
dim = 256
branch_input_dim1 = 2
branch_input_dim2 = 100
trunk_input_dim = 2

# Define the model arguments for orig_MIMONet
model_args = {
    'branch_arch_list': [
        [branch_input_dim1, 512, 512, 512, dim],
        [branch_input_dim2, 512, 512, 512, dim]
    ],
    'trunk_arch': [trunk_input_dim, 256, 256, 256, dim],
    'num_outputs': target.shape[-1] -1,  # number of output channels
    'activation_fn': nn.ReLU,
    'merge_type': 'mul',
}

In [ ]:
trunk_input_dim

2

### Load

In [ ]:
model = MIMONet(**model_args).to(device)
# Load the trained model
model_path = os.path.join(working_dir, "checkpoints/custom_best_model_lambda_1E-4.pt")

if os.path.exists(model_path):
    model.load_state_dict(torch.load(model_path, map_location=device))
    print("Model loaded successfully from", model_path)
else:
    print("Model file not found at", model_path)
    sys.exit(1)
    
# Evaluate the model on the test set
model.eval()

# print the number of parameters in the model
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Number of trainable parameters in the model: {num_params}")

Model loaded successfully from /projects/bcnx/kazumak2/MIMONet/HeatExchanger/checkpoints/custom_best_model_lambda_1E-4.pt
Number of trainable parameters in the model: 1762052


/tmp/ipykernel_301297/821581263.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=device))


## How to predict?

In [ ]:
# feed the test loader to the model (and save predictions and targets)
predictions = []
targets = []
with torch.no_grad():
    for batch in test_loader:
        branch_data, trunk_data, target_data = batch
        #print(branch_data)
        #print(trunk_data.shape, target_data.shape)
        branch_data = [b.to(device) for b in branch_data]
        trunk_data = trunk_data.to(device)
        target_data = target_data.to(device)
        print("target_data shape:", target_data.shape)

        output = model(branch_data, trunk_data)
        print("Output shape:", output.shape)
        predictions.append(output.cpu().numpy())
        targets.append(target_data.cpu().numpy())
        
# Convert predictions and targets to numpy arrays
predictions = np.concatenate(predictions, axis=0)
targets = np.concatenate(targets, axis=0)

# Reverse the scaling for the target data
predictions = scaler.inverse_transform(predictions)
targets = scaler.inverse_transform(targets)

target_data shape: torch.Size([1, 3977, 5])
Output shape: torch.Size([1, 3977, 4])
target_data shape: torch.Size([1, 3977, 5])
Output shape: torch.Size([1, 3977, 4])
target_data shape: torch.Size([1, 3977, 5])
Output shape: torch.Size([1, 3977, 4])
target_data shape: torch.Size([1, 3977, 5])
Output shape: torch.Size([1, 3977, 4])
target_data shape: torch.Size([1, 3977, 5])
Output shape: torch.Size([1, 3977, 4])
target_data shape: torch.Size([1, 3977, 5])
Output shape: torch.Size([1, 3977, 4])
target_data shape: torch.Size([1, 3977, 5])
Output shape: torch.Size([1, 3977, 4])
target_data shape: torch.Size([1, 3977, 5])
Output shape: torch.Size([1, 3977, 4])
target_data shape: torch.Size([1, 3977, 5])
Output shape: torch.Size([1, 3977, 4])
target_data shape: torch.Size([1, 3977, 5])
Output shape: torch.Size([1, 3977, 4])
target_data shape: torch.Size([1, 3977, 5])
Output shape: torch.Size([1, 3977, 4])
target_data shape: torch.Size([1, 3977, 5])
Output shape: torch.Size([1, 3977, 4])
targ

(Evaluate)

In [ ]:
# Compute L2 norm over grid points for each sample and channel
l2_pred = np.linalg.norm(predictions, axis=1)  # shape: (samples, channels)
l2_gt = np.linalg.norm(targets[..., :4], axis=1)  # shape: (samples, channels)

print("L2 norm of predictions shape:", l2_pred.shape)
print("L2 norm of targets shape:", l2_gt.shape)


# Compute L2 error over grid points for each sample and channel
l2_err = np.linalg.norm(predictions - targets[..., :4], axis=1)  # shape: (samples, channels)

# Compute relative error (avoid division by zero)
rel_err = l2_err / (l2_gt + 1e-8)  # shape: (samples, channels)

# Mean over samples for each channel
mean_rel_err_per_channel = np.mean(rel_err, axis=0)  # shape: (channels,)

print("Mean relative L2 error per channel:", mean_rel_err_per_channel * 100, "%")

# standard deviation of relative error per channel
std_rel_err_per_channel = np.std(rel_err, axis=0)  # shape: (channels,)

print("Standard deviation of relative L2 error per channel:", std_rel_err_per_channel)

L2 norm of predictions shape: (310, 4)
L2 norm of targets shape: (310, 4)
Mean relative L2 error per channel: [0.7996377 1.4521604 1.0151619 0.5177918] %
Standard deviation of relative L2 error per channel: [0.00014584 0.0002911  0.0002742  0.00038389]


In [ ]:
!pip install torchvision

In [ ]:
import os
import sys
import numpy as np
import argparse
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

#import torchvision
#import torchvision.transforms as transforms
from torch.autograd import Variable

In [ ]:
import argparse
import numpy as np
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

# Local file copied from the original repo – keep the path if needed.
from differential_evolution import differential_evolution  # noqa: E402




# ──────────────────────────────────────────────────────────────────────────────
# Utilities
# ──────────────────────────────────────────────────────────────────────────────

'''def perturb_vector(xs: np.ndarray, base_vec: torch.Tensor) -> torch.Tensor:
    """Apply a batch of perturbations *xs* to *base_vec*.

    Parameters
    ----------
    xs : ndarray, shape (n_pop, 2*k) or (2*k,)
        Genome(s) produced by DE: (idx₀, val₀, …).
    base_vec : Tensor, shape (input_dim,)
        Unperturbed input.

    Returns
    -------
    Tensor, shape (n_pop, input_dim)
        Batch of perturbed inputs ready for the model.
    """
    if xs.ndim == 1:
        xs = np.expand_dims(xs, 0)
    n_pop, genome_len = xs.shape
    k = genome_len // 2

    # Duplicate the base vector n_pop times – stays on CPU for now.
    vecs = base_vec.repeat(n_pop, 1)

    for row, genome in enumerate(xs):
        for j in range(k):
            idx = int(round(genome[2 * j]))  # ensure integer index
            val = genome[2 * j + 1]
            idx_clamped = max(0, min(idx, base_vec.numel() - 1))
            vecs[row, idx_clamped] = val
    return vecs'''
def perturb_vector(xs: np.ndarray, base_vec: torch.Tensor, order: str = "2,100"):
    """
    Apply k-feature perturbations and split the result into [branch1, branch2].

    Parameters
    ----------
    xs : ndarray, shape (pop, 2*k) or (2*k,)
        Genome(s): (idx₀, val₀, idx₁, val₁, ...).
    base_vec : Tensor, shape (102,)
        Original input.
    order : str
        "100,2"  → first 100 elements belong to branch **2**, last 2 to branch **1**.
        "2,100" → opposite.

    Returns
    -------
    list[Tensor] – [branch1, branch2]
        branch1: (pop, 2)   branch2: (pop, 100)
    """
    if xs.ndim == 1:
        xs = xs[None]

    pop, g_len = xs.shape
    k = g_len // 2

    # duplicate base
    vecs = base_vec.repeat(pop, 1)

    # overwrite selected indices
    for r, genome in enumerate(xs):
        for j in range(k):
            idx = int(round(genome[2 * j]))
            idx = max(0, min(idx, 101))
            vecs[r, idx] = genome[2 * j + 1]

    # split into branches
    first_len, second_len = map(int, order.split(","))
    first, second = vecs[:, :first_len], vecs[:, first_len:]

    if first_len == 100:
        branch2, branch1 = first, second
    else:
        branch1, branch2 = first, second

    return [branch1, branch2]


def relative_l2_error(pred: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
    """Compute ‖pred ‑ target‖₂ / ‖target‖₂ along dim=1."""
    pred_flat = pred.view(pred.size(0), -1)        # (B, N)
    tgt_flat  = target.view(target.size(0),  -1)         # (1 or B, N)
    diff_norm = torch.norm(pred_flat - tgt_flat, dim=1)      # (B,)
    tgt_norm  = torch.norm(tgt_flat,           dim=1).clamp_min(1e-12)
    return diff_norm / tgt_norm                 # (B,)


# ──────────────────────────────────────────────────────────────────────────────
# Attack core
# ──────────────────────────────────────────────────────────────────────────────

def de_objective(xs: np.ndarray, trunk : torch.Tensor ,base_x: torch.Tensor, y_true: torch.Tensor, model: torch.nn.Module,
                 device: torch.device, maximize: bool = True) -> np.ndarray:
    """Vectorised objective for DE.  Returns *negative* error (to minimise)."""
    #print(perturb_vector(xs, base_x))
    perturbed = perturb_vector(xs, base_x)
    #print("perturbed shape:", perturbed[0].shape, perturbed[1].shape)
    #print(trunk.shape)
    N = perturbed[0].shape[0]
    M = trunk.shape[1]
    trunk_repeated = np.repeat(trunk.unsqueeze(0), N, axis=0)  # shape: (N, M)
    #print("trunk_repeated shape:", trunk_repeated.shape)
    branch_data = [b.to(device) for b in perturbed]
    trunk_data = trunk_repeated.to(device)
    with torch.no_grad():
        y_pred = model(branch_data,trunk_data)
        #print(y_true.shape)
        #print("y_pred shape:", y_pred.shape)
    err = relative_l2_error(y_pred, y_true[..., :-1].to(device)).cpu().numpy()  # shape (n_pop,)
    return -err if maximize else err

In [ ]:
def attack_single(base_x: torch.Tensor, trunk : torch.Tensor,y_true: torch.Tensor, model: torch.nn.Module,
                  features: int, error_thr: float, val_min: float, val_max: float,
                  maxiter: int, popsize: int, device: torch.device, verbose: bool = False):
    """Run DE attack on a single (x, y) pair.  Returns (success, best_genome)."""
    input_dim = base_x.numel()
    bounds = []
    for _ in range(features):
        bounds.append((0, input_dim - 1))      # index
        bounds.append((val_min, val_max))      # new value

    # Pop‑multiplier like original code (total pop = popsize * n_params)
    popmul = max(1, popsize)

    predict_fn = lambda xs: de_objective(xs, trunk ,base_x, y_true, model, device, True)

    def callback_fn(genome, convergence):
        # Early stop if success achieved
        #print(perturb_vector(genome, base_x))
        perturbed = perturb_vector(genome, base_x)
        N = perturbed[0].shape[0]
        M = trunk.shape[1]
        trunk_repeated = np.repeat(trunk.unsqueeze(0), N, axis=0)
        #print(trunk.shape)
        #print("trunk_repeated shape:", trunk_repeated.shape)
        branch_data = [b.to(device) for b in perturbed]
        trunk_data = trunk_repeated.to(device)
        #print(y_true.shape, y_true.to(device).shape)
        with torch.no_grad():
            err = relative_l2_error(model(branch_data,trunk_data), y_true[..., :-1].to(device))[0].item()
        if verbose:
            #print(f"  Current best error = {err:.3f}")
            return err > error_thr

    # Optional initial population: random indices + gaussian values
    n_params = 2 * features
    inits = np.zeros((popmul * n_params, n_params))
    for row in inits:
        for f in range(features):
            row[2 * f] = np.random.randint(0, input_dim)
            row[2 * f + 1] = np.random.uniform(val_min, val_max)

    result = differential_evolution(
        predict_fn,
        bounds,
        maxiter=maxiter,
        popsize=popmul,
        recombination=1.0,
        atol=-1,
        init=inits,
        polish=False,
        callback=callback_fn,
        disp=False,
    )

    # Evaluate final error
    final_vec = perturb_vector(result.x, base_x)
    N = 1
    trunk_repeated = np.repeat(trunk.unsqueeze(0), N, axis=0)
    branch_data = [b.to(device) for b in final_vec]
    trunk_data = trunk_repeated.to(device)
    print("Final perturbation vector:", final_vec)
    with torch.no_grad():
        final_err = relative_l2_error(model(branch_data,trunk_data), y_true[..., :-1].to(device))[0].item()
    success = final_err > error_thr
    return success, result.x, final_err

IndentationError: expected an indented block (3302639600.py, line 32)

In [ ]:
# Create a torch tensor of size 2 and another of size 100
a = torch.randn(2)
b = torch.randn(100)

print(a)
print(b)
# Concatenate them into a single tensor of size 102
c = torch.cat([a, b])
print(c)
print("Concatenated shape:", c.shape)   
# Separate them back into the original shapes
a_recovered = c[:2]
b_recovered = c[2:]

print("Recovered tensors:")
print("a:", a_recovered)
print("b:", b_recovered)
print("a shape:", a_recovered.shape)
print("b shape:", b_recovered.shape)

tensor([ 0.9422, -0.0427])
tensor([-2.0045e-01, -1.9188e+00, -2.7783e+00, -6.3558e-01,  1.3906e+00,
        -7.9101e-01,  6.6308e-02, -1.8656e-01, -3.2129e-01,  5.5395e-01,
         1.1021e+00, -1.1279e+00, -6.8353e-01,  2.9178e-01, -2.1601e-01,
         5.2136e-01, -6.4081e-01, -4.0850e-01, -8.0916e-01,  6.2264e-01,
        -2.6245e-02,  1.2133e+00,  6.8626e-01,  8.4702e-01, -6.2899e-01,
        -2.3270e+00,  8.0562e-01, -1.5677e-01, -2.7960e-01,  5.8437e-01,
        -2.3964e+00,  2.1479e+00,  6.4548e-01, -3.3061e-01,  5.1956e-01,
        -2.1514e-01, -1.0601e+00, -1.0424e+00, -4.1656e-01,  2.7636e+00,
        -7.5705e-01,  2.3793e-02,  1.2877e-03,  1.7885e+00, -1.1677e+00,
        -1.8478e-01, -2.9311e-01,  3.1278e-01, -1.2002e+00, -3.6185e-01,
        -9.7849e-01, -4.5705e-01,  2.5498e-02,  4.1754e-02,  1.0996e+00,
         1.7612e-01,  1.2825e+00, -1.9004e+00, -5.5521e-01, -7.2479e-01,
        -1.7495e+00, -6.1703e-01, -1.5411e-01,  6.3489e-01, -1.3116e-01,
         1.1921e-01, -1.

In [ ]:
torch.manual_seed(0)

In [ ]:
branch_order = (2, 100)   # (len_first, len_second)  (100→branch2 first)
features   = 10       # k — how many components attacker may change
error_thr  = 0.10        # success threshold (relative ℓ₂ error)
val_min    = -1.0        # allowed range for perturbed value
val_max    =  1.0
samples    = 20          # how many dataset rows to attack
maxiter    = 150
popsize    = 40

seed       = 0
device     = "cuda" if torch.cuda.is_available() else "cpu"
verbose    = False

np.random.seed(seed)

device = torch.device(device)

# Load model – works for either full pickled model or state_dict
model_obj = model
model = model_obj if isinstance(model_obj, torch.nn.Module) else None
if model is None:
    raise ValueError("Provide a saved *model object*, not just a state_dict – ease of use.")
model.eval()

# TODO: replace RegressionDataset with your own data loader
#dataset = RegressionDataset(n_samples=args.samples, seed=args.seed)
loader = train_loader

successes = 0
total = 0

for batch in train_loader:
    branch_data, trunk_data, target_data = batch
    b1 = branch_data[0].squeeze(0)
    b2 = branch_data[1].squeeze(0)
    x = torch.cat([b1,b2])  # assuming branch_data is a list of tensors
    trunk_data = trunk_data.squeeze(0)
    y = target_data
    success, genome, err = attack_single(
        x.cpu(),trunk_data.cpu(), y.cpu(),model,
        features=5,
        error_thr=error_thr,
        val_min=val_min,
        val_max=val_max,
        maxiter=maxiter,
        popsize=popsize,
        device=device,
        verbose=verbose,
    )

    total += 1
    successes += int(success)
    status = "✔" if success else "✘"
    #print(f"[{i+1:03d}] {status}  final error = {err:.3f}  success‑rate = {successes / total:.3%}")

print(f"\nFinal success rate over {total} samples: {successes / total:.2%}")




[tensor([[ 0.2700,  0.6351],
        [ 0.2700,  0.6351],
        [ 0.2700,  0.6351],
        [ 0.2700,  0.6351],
        [-0.3517,  0.6351],
        [ 0.2700,  0.3953],
        [ 0.2700,  0.6351],
        [ 0.2700,  0.6351],
        [ 0.2457,  0.6351],
        [-0.1166,  0.6351],
        [ 0.2700,  0.6351],
        [-0.4705,  0.6351],
        [ 0.2700,  0.6351],
        [ 0.2700,  0.6351],
        [ 0.2700,  0.6351],
        [ 0.2700,  0.6351],
        [ 0.2700,  0.6351],
        [ 0.2700,  0.6351],
        [ 0.2700,  0.6351],
        [ 0.2700,  0.6351],
        [ 0.2700,  0.6351],
        [ 0.2700,  0.6351],
        [ 0.2700,  0.5673],
        [ 0.2700,  0.6351],
        [ 0.2700,  0.6351],
        [ 0.2700,  0.6351],
        [ 0.2700,  0.6351],
        [ 0.2700,  0.6351],
        [ 0.2700,  0.6351],
        [ 0.2700,  0.6351],
        [ 0.2700,  0.6351],
        [ 0.2700,  0.6351],
        [ 0.2700,  0.6351],
        [ 0.2700,  0.6351],
        [ 0.2700,  0.6351],
        [ 0.2700,  

[tensor([[-0.9805,  0.6351]]), tensor([[-1.9972, -1.8958, -1.7946, -1.6935,  0.0069, -1.4925, -1.3926, -1.2934,
         -1.1949, -1.0972, -1.0004,  0.9183, -0.8100, -0.7165, -0.6243, -0.5334,
         -0.4441, -0.3563, -0.2702, -0.1858, -0.1032, -0.0225,  0.0561,  0.1327,
          0.2072,  0.2794,  0.3494,  0.4170,  0.7154,  0.5448,  0.6049,  0.6624,
          0.7172,  0.7692,  0.8185,  0.8650,  0.9085,  0.9492,  0.9868,  1.0215,
          1.0531,  1.0817,  1.1072,  1.1295,  1.1487,  1.1647,  1.1775,  1.1871,
          1.1936,  1.1968,  1.1968,  1.1936,  1.1871,  1.1775,  1.1647,  1.1487,
          1.1295,  1.1072,  0.4628,  1.0531,  1.0215,  0.9868,  0.9492,  0.9085,
          0.8650,  0.8185,  0.7692,  0.7172,  0.6624,  0.6049,  0.5448,  0.4821,
          0.4170,  0.3494,  0.2794,  0.2072,  0.1327,  0.0561, -0.0225, -0.1032,
         -0.1858, -0.2702, -0.3563, -0.4441, -0.5334, -0.6243, -0.7165, -0.8100,
         -0.9046, -1.0004, -1.0972, -1.1949, -1.2934, -1.3926, -1.4925, -1.592

NameError: name 'i' is not defined

In [ ]:
successes

1